In [1]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

/home/adrian/anaconda3/envs/VE/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
from fastscript import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *
import warnings

from charades import *

ModuleNotFoundError: No module named 'video_block'

In [3]:
#exporti
@call_parse
def main(gpu    :Param("GPU to run on", int)=None,
         eugenio:Param("Variable to control whether is Eugenio or me that's calling the training", bool)=False,
         n_lbl  :Param("# of different labels per batch", int)=8,
         n_el   :Param("# of elements per label", int)=4,
         l      :Param("Target number of frames of the ResizeTime transform", int)=60,
         sz     :Param("size for Resize", int)=224,
         n_epoch:Param("# of epochs to train", int)=8,
        ):
        
    if gpu is not None: 
        gpu = setup_distrib(gpu)    
        items = rank0_first(lambda: read_data(eugenio)) 
    else:
        items = read_data(eugenio)

    learn = get_learner(items, l, sz, n_lbl, n_el)
    
    
    torch.cuda.empty_cache()
    if gpu is not None:
        print("Distributed Data Parallel training started")
        with learn.distrib_ctx(gpu):
            learn.fine_tune(n_epoch)
    else:
        warnings.filterwarnings("ignore", message='.*nonzero.*', category=UserWarning)
        print("Data Parallel training started")
        with learn.parallel_ctx(device_ids=[0,1]):
            learn.fine_tune(n_epoch)

    learn.save('trained_models/Inflated1')

In [13]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ufc.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.


In [6]:
# default_exp first_inflated_NN